<a href="https://colab.research.google.com/github/ibadrather/pytorch_learn/blob/main/Part%2013%20-%20Multivariate_Time_Series_Data_Preprocessing_with_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip freeze

In [7]:
!pip install --quiet pytorch-lightning
!pip install --quiet tqdm